# 패키지 다운로드

In [1]:
# pip install xarray

# 패키지 로딩

In [8]:
import xarray as xr
import pandas as pd

# nc -> csv 변환

In [7]:
ds = xr.open_dataset('./data/EN.4.2.2_D20_SCTR_f.nc')

df = ds.D20_SCTR.to_dataframe()

df.reset_index(inplace=True)

df.to_csv('./output/SCTR.csv', index=False)

ds.close()

# 종속변수 불러오기

In [13]:
sctr = pd.read_csv("./output/SCTR.csv")

sctr["TIME1"] = pd.to_datetime(sctr["TIME1"])

sctr["TIME1"] = sctr["TIME1"].dt.normalize()

,TIME1,D20_SCTR
0,1991-01-16 12:00:00,54.960640
1,1991-02-15 00:00:00,59.899442
2,1991-03-16 12:00:00,70.225124
3,1991-04-16 00:00:00,67.375125
4,1991-05-16 12:00:00,63.783066
...,...,...
395,2023-12-16 12:00:00,124.376897
396,2024-01-16 12:00:00,117.300079
397,2024-02-15 12:00:00,121.968669
398,2024-03-16 12:00:00,118.355688
